In [124]:
from solver import Solver 
import pennylane as qml 
from pennylane import numpy as np
from preparator import Preparator

In [243]:
def _stiff_boost (Q : np.ndarray, alpha : float = 100.0):
      

        S = np.linalg.eig(Q)[1].T

        stiffed_diag = np.linalg.eigvals(Q)
        mini = 0 
        maxi = 0

        for i in range (len(stiffed_diag)): 

            if (np.allclose(np.abs(stiffed_diag[i]),np.max(np.abs(stiffed_diag)))):
                maxi = i
                stiffed_diag[i] *= alpha 
            if (np.allclose(np.abs(stiffed_diag[i]),np.min(np.abs(stiffed_diag)))):
                mini = i
                stiffed_diag[i] /= alpha


        D = np.diag(stiffed_diag)

        T = np.diag([1.0] * S.shape[0])

        T[mini,mini] = T[mini,mini] / np.sqrt(alpha)
        T[maxi, maxi] = T[maxi,maxi] * np.sqrt(alpha) 

        Qs = np.linalg.inv(S @ T).T @ D @ np.linalg.inv(S @ T) 

        return Qs


In [242]:


prep = Preparator()

dev = qml.device('default.qubit', wires = 10, shots = 1000)
sol = Solver (dev = dev, gate = 'X')

Q = np.random.uniform(low = -1, high = 1, size = (10,10)) 
Q = (Q.T + Q)/2
print(np.linalg.eigvals(Q))
#Q = prep.stiffnessPreparator(size = 10, stiffness = 100)

eigs = np.abs(np.linalg.eigvals(Q))

print(np.max(eigs) / np.min(eigs))

Qstiffed = _stiff_boost(Q, alpha = 10)

eigs = np.abs(np.linalg.eigvals(Qstiffed))

print(np.max(eigs) / np.min(eigs))
print(sol.checkQUBO(Q))
print(sol.checkQUBO(Qstiffed))

[-2.37419683 -2.05774502 -1.64242804 -0.73824376 -0.18529611  0.65084064
  0.88330629  1.91290143  1.76424631  1.42440021]
12.812987786417045
12.812987786417038
(tensor(-3.09334025, requires_grad=True), [tensor([1, 0, 1, 0, 0, 1, 1, 0, 1, 1], requires_grad=True)], 0.15245890617370605)
(tensor(-0.4307361, requires_grad=True), [tensor([1, 0, 1, 0, 0, 1, 1, 1, 1, 0], requires_grad=True)], 0.14336729049682617)


(tensor(-2.8361001, requires_grad=True), [tensor([1, 0, 1, 0, 1, 1, 1, 0, 1, 0], requires_grad=True)], 0.16142797470092773)
(tensor(-0.00152623, requires_grad=True), [tensor([0, 0, 0, 1, 1, 1, 1, 0, 1, 0], requires_grad=True)], 0.16728591918945312)
